In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

'''reading from Database using pandas'''
xls = pd.ExcelFile('LSVT_voice_rehabilitation.xlsx')
df_data = pd.read_excel(xls, 'Data')
df_target = pd.read_excel(xls, 'Binary response')
df_demographics = pd.read_excel(xls, 'Subject demographics')

'''10-fold cross validation '''

X = df_data.values
y = df_target.values

X_folds = np.array_split(X, 10)
y_folds = np.array_split(y, 10)


In [6]:
'''Support vector machine implementation using 10-fold '''
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

def SVMClassification(svc):
    scores = list()
    f1_scores = list()
    for k in range(10):
    
        X_train = list(X_folds)
        X_test = X_train.pop(k)
        X_train = np.concatenate(X_train)

        y_train = list(y_folds)
        y_test = y_train.pop(k)
        y_train = np.concatenate(y_train)

        svc = make_pipeline(StandardScaler(), svc)
        model = svc.fit(X_train,np.ravel(y_train))
        scores.append(model.score(X_test, y_test))

        y_pred = model.predict(X_test)
        f1_scores.append(f1_score(y_test[:,0], y_pred, average='micro'))
        
    avg_scores = sum(scores)/10
    avg_f1 = sum(f1_scores)/10
    
    return (avg_scores,avg_f1)


# Notes:

## 1. StandardScaler 
The main idea is to normalize/standardize i.e. μ = 0 and σ = 1 your features/variables/columns of X, individually, before applying any machine learning model. Thus, StandardScaler() will normalize the features i.e. each column of X, INDIVIDUALLY so that each column/feature/variable will have μ = 0 and σ = 1.

Note that non-tree models models such as SVM, LDA etc. are often hugely dependent on normalization.

## 2. micro
micro parameter in f1-score Calculate metrics globally by counting the total true positives, false negatives and false positives.

# Linear

In [21]:
svc = svm.SVC(kernel = 'linear',max_iter=10000)
(avg_scores,avg_f1) = SVMClassification(svc)

print('Linear accuracy    ' + str(avg_scores))
print('Linear F1-measure  ' + str(avg_f1))

Linear accuracy    0.7852564102564104
Linear F1-measure  0.7852564102564104


## Polynomial 

In [22]:
r=[-.5, 0, .5,1 ]
d=[2,3,4]

import itertools
for r,d in itertools.product(r,d):
    svc = svm.SVC(kernel = 'poly',max_iter=10000,degree = d, coef0 = r)
    (avg_scores,avg_f1) = SVMClassification(svc)
    
    print('Polynomial accuracy(d={},r={})    '.format(d,r) +str(avg_scores) )
    print('Polynomial f1-measure(d={},r={})  '.format(d,r) +str(avg_f1) +'\n')

Polynomial accuracy(d=2,r=-0.5)    0.45576923076923076
Polynomial f1-measure(d=2,r=-0.5)  0.45576923076923076

Polynomial accuracy(d=3,r=-0.5)    0.7442307692307693
Polynomial f1-measure(d=3,r=-0.5)  0.7442307692307693

Polynomial accuracy(d=4,r=-0.5)    0.4397435897435898
Polynomial f1-measure(d=4,r=-0.5)  0.4397435897435898

Polynomial accuracy(d=2,r=0)    0.6923076923076923
Polynomial f1-measure(d=2,r=0)  0.6923076923076923

Polynomial accuracy(d=3,r=0)    0.6916666666666667
Polynomial f1-measure(d=3,r=0)  0.6916666666666667

Polynomial accuracy(d=4,r=0)    0.6512820512820514
Polynomial f1-measure(d=4,r=0)  0.6512820512820514

Polynomial accuracy(d=2,r=0.5)    0.8282051282051283
Polynomial f1-measure(d=2,r=0.5)  0.8282051282051283

Polynomial accuracy(d=3,r=0.5)    0.8275641025641025
Polynomial f1-measure(d=3,r=0.5)  0.8275641025641025

Polynomial accuracy(d=4,r=0.5)    0.8198717948717948
Polynomial f1-measure(d=4,r=0.5)  0.8198717948717948

Polynomial accuracy(d=2,r=1)    0.8673076

## degree 
It appears that the degree parameter controls the flexibility of the decision boundary. Higher degree kernels yield a more flexible decision boundary.

## coef0
Independent term in kernel function.it is constant parameter in kernel formula. By adjusting it correctly, the accuracy of the model can be improved

# Gussian RBF SVM parameters

In [23]:
gamma = [0.001 , 0.005,0.01,0.15]
for g in gamma:
    svc = svm.SVC(kernel = 'rbf',max_iter=10000,gamma = g)
    (avg_scores,avg_f1) = SVMClassification(svc)
    
    print('RBF accuracy(gamma{})    '.format(g) +str(avg_scores) )
    print('RBF f1-measure(dgamma{})  '.format(g) +str(avg_f1) +'\n')    

RBF accuracy(gamma0.001)    0.8282051282051281
RBF f1-measure(dgamma0.001)  0.8282051282051281

RBF accuracy(gamma0.005)    0.8423076923076923
RBF f1-measure(dgamma0.005)  0.8423076923076923

RBF accuracy(gamma0.01)    0.8339743589743589
RBF f1-measure(dgamma0.01)  0.8339743589743589

RBF accuracy(gamma0.15)    0.6666666666666667
RBF f1-measure(dgamma0.15)  0.6666666666666667



## gamma
Intuitively, the gamma parameter defines how far the influence of a single training example reaches, with low values meaning 'far' and high values meaning 'close'. The gamma parameters can be seen as the inverse of the radius of influence of samples selected by the model as support vectors.

The behavior of the model is very sensitive to the gamma parameter. If gamma is too large, the radius of the area of influence of the support vectors only includes the support vector itself and no amount of regularization with C will be able to prevent overfitting.

When gamma is very small, the model is too constrained and cannot capture the complexity or “shape” of the data. The region of influence of any selected support vector would include the whole training set. The resulting model will behave similarly to a linear model with a set of hyperplanes that separate the centers of high density of any pair of two classes.



# Sigmoid

In [29]:
r=[-.5, 0, .5,1 ]
for r in r:
    svc = svm.SVC(kernel = 'sigmoid',max_iter=10000,coef0 = r)
    (avg_scores,avg_f1) = SVMClassification(svc)
    
    print('Sigmoid accuracy(r={})    '.format(r) +str(avg_scores) )
    print('Sigmoid f1-measure(r={})  '.format(r) +str(avg_f1) +'\n')

Sigmoid accuracy(r=-0.5)    0.8435897435897436
Sigmoid f1-measure(r=-0.5)  0.8435897435897436

Sigmoid accuracy(r=0)    0.8589743589743591
Sigmoid f1-measure(r=0)  0.8589743589743591

Sigmoid accuracy(r=0.5)    0.8743589743589745
Sigmoid f1-measure(r=0.5)  0.8743589743589745

Sigmoid accuracy(r=1)    0.7480769230769232
Sigmoid f1-measure(r=1)  0.7480769230769232



# conclusion
based on resuls, the best model is Sigmoid Kernel with r=0.5 which result is 87 percent model accuracy